In [108]:
import os
import random
import csv
from pathlib import Path
import random
import re
import pandas as pd
import numpy as np
from unidecode import unidecode
from nltk.corpus import stopwords
import spacy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------------------
# 1. Parameters, paths and models
# ----------------------------------------

# Path & Parameters 
project_root = Path().resolve().parent
attribute_file = project_root / "data" / "attribute_base_processedV2.xlsx"
jocas_dir = "/Users/alfonso/Desktop/JOCAS"  # adjust if needed
output_file = Path(project_root) / "data" / "attribute_base_processedV3.xlsx"

# Parameters
SAMPLE_SIZE = 100000
RANDOM_SEED = 42

# Minimal cleaning + unaccenting
def clean_text(text: str) -> str:
    s = re.sub(r"http\S+|www\.\S+", " ", text)
    return re.sub(r"\s+", " ", s).strip()

# Load SpaCy with parser 
nlp = spacy.load("fr_core_news_sm", disable=["ner"])

# Load SBERT 
sbert = SentenceTransformer("distiluse-base-multilingual-cased-v2")

# Stop-words 
french_stopwords = set(stopwords.words("french"))


# ----------------------------------------
# 2. Build seed-token sets per category
# ----------------------------------------
attribute_df = (
    pd.read_excel(attribute_file)
      .loc[:, ["attribute_category_fr", "expression"]]
      .dropna()
)

cat_to_seeds = {}
for cat, grp in attribute_df.groupby("attribute_category_fr"):
    seeds = set()
    for expr in grp["expression"].astype(str):
        toks = clean_text(expr).split()
        seeds.update([tok for tok in toks if tok not in french_stopwords])
    cat_to_seeds[cat] = seeds

In [6]:
# ----------------------------------------
# 3. Ingest & Aggregate JOCAS Job Ads
# ----------------------------------------

# Initialize container
jocas_list = []


# Walk though source folders
for source_folder in os.listdir(jocas_dir):
    source_path = os.path.join(jocas_dir, source_folder)
    
    if os.path.isdir(source_path):
        # Collect all csv files under this source
        csv_files = []
        for root, dirs, files in os.walk(source_path):
            for file in files:
                
                # Case-insensitive check for .csv files
                if file.lower().endswith('.csv'):
                    csv_files.append(os.path.join(root, file))
        
        print(f"Found {len(csv_files)} CSV files in source folder: {source_folder}")
        
        if not csv_files:
            continue
        
        # Select up to 50 csv files from this source
        if len(csv_files) > 50:
            selected_files = random.sample(csv_files, 50)
        else:
            selected_files = csv_files
        
        print(f"Processing {len(selected_files)} files from source folder: {source_folder}")
        
        # Process each selected CSV file
        for file_path in selected_files:
            try:
                df = pd.read_csv(
                    file_path,
                    header=0,
                    sep=';',
                    on_bad_lines='skip',
                    low_memory=False,
                    quoting=csv.QUOTE_MINIMAL,
                    escapechar='\\'
                )
                jocas_list.append(df)
            except Exception as e:
                print(f"Failed to read {file_path}: {e}")

        print(f"Finished processing source folder: {source_folder}")

# Aggregate all dataframes into one
df_job = pd.concat(jocas_list, ignore_index=True)
print(f"Total job ads: {len(df_job)}")


Found 361 CSV files in source folder: regionsjob_nord
Processing 50 files from source folder: regionsjob_nord
Finished processing source folder: regionsjob_nord
Found 357 CSV files in source folder: regionsjob_paris
Processing 50 files from source folder: regionsjob_paris
Finished processing source folder: regionsjob_paris
Found 359 CSV files in source folder: regionsjob_rhonealpes
Processing 50 files from source folder: regionsjob_rhonealpes
Finished processing source folder: regionsjob_rhonealpes
Found 362 CSV files in source folder: apec
Processing 50 files from source folder: apec
Finished processing source folder: apec
Found 169 CSV files in source folder: keljob
Processing 50 files from source folder: keljob
Finished processing source folder: keljob
Found 360 CSV files in source folder: regionsjob_sudouest
Processing 50 files from source folder: regionsjob_sudouest
Finished processing source folder: regionsjob_sudouest
Found 361 CSV files in source folder: regionsjob_paca
Process

/var/folders/5z/n4wfh12s55d5j381j6gl47c40000gn/T/ipykernel_7007/3032757085.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_job = pd.concat(jocas_list, ignore_index=True)


Total job ads: 2017548


In [50]:
# ----------------------------------------
# 4. Sample & clean job descriptions
# ----------------------------------------
raw = (
    df_job["description_full"]
      .dropna()
      .astype(str)
      .sample(n=SAMPLE_SIZE, random_state=RANDOM_SEED)
      .tolist()
)
cleaned_texts = [clean_text(t) for t in raw]

# ----------------------------------------
# 5. Extract noun-chunks as candidates
# ----------------------------------------
candidate_phrases = {cat: set() for cat in cat_to_seeds}
for sent in cleaned_texts:
    doc = nlp(sent)
    for chunk in doc.noun_chunks:
        phrase = clean_text(chunk.text)
        toks = phrase.split()
        # keep only chunks of length 3–7
        if 3 <= len(toks) <= 7:
            for cat, seeds in cat_to_seeds.items():
                if any(tok in seeds for tok in toks):
                    candidate_phrases[cat].add(phrase)

# debug
for cat, phs in candidate_phrases.items():
    print(f"{cat}: {len(phs)} candidates")

# ----------------------------------------
# 6. Compute category centroids
# ----------------------------------------
cat_centroids = {}
for cat, grp in attribute_df.groupby("attribute_category_fr"):
    exprs = grp["expression"].astype(str).tolist()
    exprs_clean = [clean_text(e) for e in exprs]
    emb = sbert.encode(exprs_clean, convert_to_numpy=True)
    cat_centroids[cat] = emb.mean(axis=0, keepdims=True)

# ----------------------------------------
# 7. Score & select top-30 per category
# ----------------------------------------
results = []
for cat, centroid in cat_centroids.items():
    cands = list(candidate_phrases[cat])
    if not cands:
        continue
    cand_embs = sbert.encode(cands, convert_to_numpy=True)
    sims = cosine_similarity(centroid, cand_embs)[0]
    top_idx = np.argsort(sims)[::-1][:30]
    for idx in top_idx:
        results.append({
            "attribute_category_fr": cat,
            "suggested_expression": cands[idx],
            "similarity": float(sims[idx])
        })

# ----------------------------------------
# 8. Preview top-5 per category
# ----------------------------------------
results_df = pd.DataFrame(results)
preview = (
    results_df
      .sort_values(["attribute_category_fr", "similarity"], ascending=[True, False])
      .groupby("attribute_category_fr")
      .head(5)
      .reset_index(drop=True)
)
print(preview.to_string(index=False))


Autonomie dans l’exécution des tâches: 8265 candidates
Bel emplacement: 5389 candidates
Bonne rémunération des heures supplémentaires: 6179 candidates
Bonnes perspectives de carrière: 11091 candidates
Bons collègues: 8582 candidates
Chalet d’entreprise: 1388 candidates
Choix entre temps plein et temps partiel: 3777 candidates
Chèques vacance: 8 candidates
Congés flexibles/étendus: 3885 candidates
Contrat à durée déterminée: 4382 candidates
Contrat à temps partiel: 708 candidates
Contrat à temps plein: 875 candidates
Emplacement central: 1715 candidates
Emploi permanent: 4227 candidates
Emploi sur appel: 4780 candidates
Emploi temporaire: 3971 candidates
Environnement social: 12390 candidates
Espace de bureau partagé: 3960 candidates
Exemption des réglementations sur le temps de travail: 9922 candidates
Formation en cours d’emploi: 12260 candidates
Heures supplémentaires requises: 9429 candidates
Horaire de travail de jour régulier: 3609 candidates
Niveau de rémunération: 6287 candidate

In [123]:
## We extract this outcome with all expressions retained 
# word_embedding = project_root / "data" / "word_embedding_expressions.xlsx"
# results_df.to_excel(word_embedding)

# ----------------------------------------
# We manually selected relevant expressions (=1 in the added "keep" column, 0 otherwise)
# We need to reimport this excel and continue our worklow with it 
# --> First we lemmatize and clean those new expressions with the cleaning function
# --> We check if there are useless words that we could add in the stop list
# --> we take unique lemmatized expressions and add them (with their row version) in the 
#     attribute base if they do not already exist
# ----------------------------------------

# Re-Load updated dataset
new_expr = pd.read_excel(word_embedding)

# Select relevant potential expressions
new_expr = new_expr[new_expr["keep"]==1]


# ----------------------------------------
# 9.1 Build French stop-word set
# ----------------------------------------
# base NLTK list
french_stopwords = set(stopwords.words('french'))

# extra in-between tokens
additional_stopwords = [
    'alors', 'assez', 'avoir', 'cahier', 'connaissance',
    'directement', 'essentiellement', 'ensuite', 'egalemer',
    'egalement', 'integralement', 'notamment', 'parfaitement',
    'particulieremer', 'plus', 'principalement', 'prioritairement',
    'progressivement', 'progressivemer', 'rapidement',
    'aussi', 'ainsi', 'donc', 'constant', 'continues', 'prevue',
    'prevues', 'possibles', 'possible', 'sont', 'individualisee',
    'soutien', 'personnalise', 'veritable', 'tres',
    'tre', 'veritabl', 'extremement', 'futur', 'tous', 'quelques', 
    'uniquement'
]

french_stopwords |= set(additional_stopwords)


# ----------------------------------------
# 9.2 Cleaning + stop-word removal
# ----------------------------------------
def clean_and_remove_stopwords(text: str) -> str:
    """
    1) Drop URLs, lowercase
    2) Strip accents via unicodedata
    3) Keep only letters & spaces
    4) Collapse whitespace
    5) Remove French stop-words
    """
    # 1) URLs + lowercase
    s = re.sub(r'http\S+|www\.\S+', ' ', text).lower()
    # 2) strip accents
    s = unicodedata.normalize('NFD', s)
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    # 3) keep only letters & spaces
    s = re.sub(r'[^a-z\s]', ' ', s)
    # 4) collapse whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    # 5) remove stop-words
    tokens = [tok for tok in s.split() if tok not in french_stopwords]
    return ' '.join(tokens)


# ----------------------------------------
# 9.3 Apply to your filtered expressions
# ----------------------------------------
# assume new_expr has already been read and filtered (keep==1)
new_expr['lemma'] = new_expr['suggested_expression'] \
                       .astype(str) \
                       .apply(clean_and_remove_stopwords)


# ----------------------------------------
# 9.4 Apply to filtered expressions 
# ----------------------------------------

# Drop rows with duplicated lemma
new_expr.drop_duplicates(subset=["lemma"], inplace=True)

# Raname expression column
new_expr.rename(columns={"suggested_expression": "expression"}, inplace=True)

# Select important columns
new_expr = new_expr.loc[:, ["attribute_category_fr", "expression"]]


# ----------------------------------------
# 9.5 Append new expressions to main database
# ----------------------------------------

# Re-upload the main attribute base excel
attribute_base = pd.read_excel(attribute_file)

# 1) Prepare `new_expr` with the same columns as `attribute_base`
new_rows = new_expr.assign(aggregate_category_fr=pd.NA, _is_new=True)


# 2) Tag the originals so they sort before the new ones
orig = attribute_base.copy().assign(_is_new=False)

# 3) Concatenate, sort, and forward-fill
merged = (
    pd.concat([orig, new_rows], ignore_index=True)
      .sort_values(["attribute_category_fr", "_is_new"])
      .assign(
         aggregate_category_fr=lambda df: df
           .groupby("attribute_category_fr")["aggregate_category_fr"]
           .ffill()
      )
      .drop(columns="_is_new")
      .reset_index(drop=True)
)

attribute_base = merged

# ----------------------------------------
# 9.6 Deduplicate similar lemma from the full attribute base
# ----------------------------------------

# Assume new_expr has already been read and filtered (keep==1)
attribute_base['lemma'] = attribute_base['expression'].astype(str).apply(clean_and_remove_stopwords)

# Drop rows with duplicated lemma
attribute_base.drop_duplicates(subset=["lemma"], inplace=True)


# ----------------------------------------
# Export
# ----------------------------------------
# attribute_base.to_excel(output_file, index=False)